`Reading Base64 String from DB Table using langchain`

In [55]:
from langchain_community.utilities.sql_database import SQLDatabase

uri = 'mssql+pyodbc://localhost/TPMS?driver=ODBC+Driver+18+for+SQL+Server&TrustServerCertificate=yes&Encrypt=no'
db = SQLDatabase.from_uri(uri)

db_info = db.get_table_info()

In [56]:
output = db.run("select pyAttachStream from pc_data_workattach where pzInsKey='PC001'")

#### Observation:
+ It is not extracting the complete base64 string.

`Reading Base64 String from DB Table using Python ODBC`

In [ ]:
import pyodbc 

cnxn = pyodbc.connect(
    'DRIVER={ODBC Driver 18 for SQL Server};'
    'Server=localhost;'       
    'Database=TPMS;'           
    'Port=1433;'               
    'Trusted_Connection=yes;'  
    'TrustServerCertificate=yes;'
    'Encrypt=no'
)

cursor = cnxn.cursor()

attachment_base64 = list()
cursor.execute("select pyAttachStream from pc_data_workattach where pzInsKey='PC001'") 

rows = cursor.fetchall()

for row in rows:
    row_as_string = ', '.join(str(value) for value in row)
    attachment_base64.append(row_as_string)

cursor.close()
cnxn.close()

In [60]:
attachment_base64

['...']

`Creating doc from base64 string`

In [ ]:
import base64
import re

def is_base64(string):
    base64_pattern = re.compile(r'^[A-Za-z0-9+/=\s]+$')
    return base64_pattern.fullmatch(string) is not None

def extract_text_from_base64(base64_data, output_file_path=None):
    try:
        if not is_base64(base64_data.strip()):
            print("Error: The file does not contain valid Base64-encoded text.")
            return None
        decoded_data = base64.b64decode(base64_data)
        try:
            decoded_text = decoded_data.decode('utf-16')
            if output_file_path:
                with open(output_file_path, 'w') as output_file:
                    output_file.write(decoded_text)
            return decoded_text
        except UnicodeDecodeError:
            if output_file_path:
                with open(output_file_path, 'wb') as output_file:
                    output_file.write(decoded_data)
            return None

    except (base64.binascii.Error, ValueError) as e:
        print(f"Error: Failed to decode Base64 content. Details: {e}")

text = extract_text_from_base64(attachment_base64[0], "base_64_string_decode.pdf")
if text:
    print("Decoded text:")
    print(text) 

`Identifying the customer intent filter`

In [61]:
from langchain_groq import ChatGroq

llm_model = ChatGroq(
    temperature=0,
    model_name="llama-3.1-8b-instant",
    api_key="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"
)

In [63]:
from typing import List
from langchain_core.tools import tool
from pydantic import BaseModel, Field
from langchain.prompts import PromptTemplate
from langgraph.prebuilt import create_react_agent
from langchain_community.utilities.sql_database import SQLDatabase
from langchain_core.prompts import ChatPromptTemplate, SystemMessagePromptTemplate, MessagesPlaceholder

In [85]:
query = "Can you tell me about M005 milestone"

In [92]:
from typing import List
from pydantic import BaseModel, Field

class SelectedTables(BaseModel):
    """List of selected tables"""
    tables: List[str] = Field(description="List of table name")
    reasoning: str = Field(description="Reason for choosing the table")

prompt = f"""   
<|begin_of_text|>
<|start_header_id|>system<|end_header_id|>  
You are an Microsoft SQL Server expert with extensive knowledge of database schemas, relationships, and querying best practices. 
Your task is to determine the appropriate tables and fields needed to answer user queries related to a given database.

## Guidelines:
1. Analyze the Question: Understand the main entities (tables) and attributes (fields) mentioned in the user's query.
2. Identify Relevant Tables: Based on the entities in the question, choose the tables that most likely hold the data needed to answer it. 
   Consider table relationships (e.g., joins, foreign keys) and context to make your selection.
3. If you are not able to find the most relevent tables for users question then include all the available tables.
4. Make sure that you are always including 'pc_work_group' table in the result list
4. Do not create any select query in your response.
5. Do not give any additional information.
6. Sample json response format:
{{
    'tables': ['table1'],
    'reasoning': Reason for choosing the table
}}

User's Query: {query}
Database table schema: {db_info}
<|eot_id|>
<|start_header_id|>assistant<|end_header_id|>
"""

stuct_llm = llm_model.with_structured_output(SelectedTables, method="json_mode")

selection = stuct_llm.invoke(prompt)
print(selection)


tables=['pr_ADEO_TaskMana_Data_Mileston', 'pr_ADEO_TaskMana_Data_Delivera', 'pc_work_group'] reasoning='The tables pr_ADEO_TaskMana_Data_Mileston and pr_ADEO_TaskMana_Data_Delivera are relevant because they contain information about milestones and deliverables, which are likely related to the M005 milestone. The pc_work_group table is included as per the guideline to always include this table in the result list.'


In [93]:
table_schema = db.get_table_info()
selected_db_info = db.get_table_info(table_names=selection.tables)

In [94]:
class SQLQuery(BaseModel):
    """Response SQL Query"""
    query: str = Field(description="SQL query")

prompt = f"""
<|begin_of_text|>
<|start_header_id|>system<|end_header_id|>
You are an expert in Microsoft SQL Server with extensive knowledge of querying and SQL syntax. 
Your task is to generate the most appropriate SQL query to answer the user's question based on the provided selected tables and database schema.

## Guidelines:
1. Analyze the selected tables, relationships, and fields to construct a valid SQL query.
2. Use the relevant tables and include all the necessary fields in the select query to give detailed answer to the users question.
3. Ensure the query is optimized and includes necessary joins, conditions, and aggregations as needed.
4. The SQL query should be based on the user's question and the schema.
5. The response must contain only SQL query and end with semi-colon(;).
6. This SQL query response will be converted into natural response to the user, so include all the necessary columns for the table to given more information.
7. Sample json response format:
{{
    "query": "select col1, col2 from table;"
}}

User's question: {query}
Selected Tables: {selection.tables}
Database Table Schema: {selected_db_info}
<|eot_id|>
<|start_header_id|>assistant<|end_header_id|>
"""

stuct_llm = llm_model.with_structured_output(SQLQuery, method="json_mode")

sql_query = stuct_llm.invoke(prompt)
print(sql_query.query)

SELECT pr_ADEO_TaskMana_Data_Mileston.ID, pr_ADEO_TaskMana_Data_Mileston.Name, pr_ADEO_TaskMana_Data_Mileston.StartDateTime, pr_ADEO_TaskMana_Data_Mileston.EndDateTime, pr_ADEO_TaskMana_Data_Mileston.ProgressRate, pr_ADEO_TaskMana_Data_Mileston.Status, pr_ADEO_TaskMana_Data_Mileston.ActualCompletionDateTime, pr_ADEO_TaskMana_Data_Mileston.DaysTaken, pr_ADEO_TaskMana_Data_Mileston.Style, pr_ADEO_TaskMana_Data_Mileston.pySelected, pc_work_group.ProjectType, pc_work_group.CompletionRate, pc_work_group.PlannedBudget, pc_work_group.ActualBudget, pc_work_group.StartDateTime, pc_work_group.EndDateTime, pc_work_group.KPIID, pc_work_group.KPITitle, pc_work_group.ClosingReason, pc_work_group.ProjectComments, pc_work_group.RMRemarks, pc_work_group.IsDROProject, pc_work_group.DGTitle, pc_work_group.EDTitle, pc_work_group.Sector, pc_work_group.SGTitle, pc_work_group.UserDepartment, pc_work_group.Classification FROM pr_ADEO_TaskMana_Data_Mileston INNER JOIN pc_work_group ON pr_ADEO_TaskMana_Data_Mil

In [ ]:
class IntentFilter(BaseModel):
    """Query intent keys"""
    projects: List[str] = Field(description="List of project")
    tasks: List[str] = Field(description="List of task or task")
    milestone: List[str] = Field(description="List of milestone")
    delivery: List[str] = Field(description="List of delivery")


INTENT_FILTER_PROMPT = PromptTemplate(
    template="""
<|begin_of_text|>
<|start_header_id|>system<|end_header_id|>  
You are an AI expert, specialized in identifying the customer intent based on the user question and SQL query. 
The output with filters needs to specify which projects, tasks, milestones and deliverys are relevant.

Database tables schema: {schema}

Guidelines:
1. Analyze the user's question carefully.
2. Provide only the list of queries without any additional explanations or commentary.
3. The output json format should be as follows:
{{
    "projects": [<list of relevant projects>],
    "tasks": [<list of relevant tasks>],
    "milestone": [<list of relevant milestone>],
    "delivery": [<list of relevant delivery>],
}}
Example:
- If the customer wants to view data for project1, and tasks task1 and task2, the output should be formatted as:
{{
    "projects": ["project1"],
    "tasks": ["task1", "task2"]
}}
<|eot_id|>
<|start_header_id|>assistant<|end_header_id|>
User question: {user_query}
SQL query: {query}
    """,
    input_variables=["user_query", "query", "schema"],
)

stuct_llm = llm_model.with_structured_output(IntentFilter, method="json_mode")

filter_chain = INTENT_FILTER_PROMPT | stuct_llm

In [97]:
filter_chain.invoke(
    {
        "schema": table_schema,
        "user_query": query,
        "query": sql_query.query
    }
)

IntentFilter(projects=['PC005'], tasks=[], milestone=['M005'], delivery=[])

In [103]:
import pyodbc 

cnxn = pyodbc.connect(
    'DRIVER={ODBC Driver 18 for SQL Server};'
    'Server=localhost;'       
    'Database=TPMS;'           
    'Port=1433;'               
    'Trusted_Connection=yes;'  
    'TrustServerCertificate=yes;'
    'Encrypt=no'
)

cursor = cnxn.cursor()

attachment_base64 = list()
cursor.execute("select pxAttachName, pyAttachStream from pc_data_workattach where pzInsKey IN ('PC001', 'PC002')") 

rows = cursor.fetchall()

for row in rows:
    pxAttachName, pyAttachStream = row
    attachment_base64.append({"pxAttachName": pxAttachName, "pyAttachStream": pyAttachStream})

cursor.close()
cnxn.close()

In [104]:
attachment_base64

[{'pxAttachName': 'Attachment1.pdf',
  'pyAttachStream': '...'},
 {'pxAttachName': 'Attachment2.docx', 'pyAttachStream': None}]

In [110]:
decoded_data = base64.b64decode(attachment_base64[0]["pyAttachStream"])

decoded_text = decoded_data.decode('utf-8')

decoded_text

UnicodeDecodeError: 'utf-8' codec can't decode byte 0xe2 in position 10: invalid continuation byte